# Projet de fin d'étude - Parties finales (catégorisation/classification client - prédiction (ML))
## Camille Bouberka - Armand Dusart - Jules Enguehard
## encadrant : Martin Nasse
## 2020-2021 - ESME Sudria 

<p>Lors de la phase précédente (exploration de données) nous avons remarqué 2 types de factures : Les factures normales (à montant positif) et les factures annulées (à montant négatif)<p>
<p>De plus, certaines valeurs paraissent aberrantes<p>

# 2 - Traitement

In [46]:
from pandas import read_csv
df = read_csv("Online_Retail.csv", sep=";").drop("Unnamed: 0", axis=1)
print(df.count())
print(df.info())

InvoiceNo      541909
StockCode      541909
Description    540455
Quantity       541909
InvoiceDate    541909
UnitPrice      541909
CustomerID     406829
Country        541909
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


## 2.1 - Pré-traitement

On peut observer qu'il manque de valeurs, il faut donc supprimer les lignes nulles et donner le bon type aux colonnes qui en ont besoin

In [47]:
df = df.dropna()
df['CustomerID'] = df['CustomerID'].astype(int)
df.count()

InvoiceNo      406829
StockCode      406829
Description    406829
Quantity       406829
InvoiceDate    406829
UnitPrice      406829
CustomerID     406829
Country        406829
dtype: int64

## 2.2 - Ajout de la colonne CA représentant la consommation des clients (Quantity*UnitPrice)

Afin de déterminer les valeurs aberrantes, il faut créer un indicateur de consommation.

In [48]:
df['CA'] = df['Quantity'] * df['UnitPrice'] #406829
df['CA'] = df['CA'].round(2) #On arrondie à 2 chiffres après la virgule
df.describe()

,Quantity,UnitPrice,CustomerID,CA
count,406829.000000,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570,20.401854
std,248.693370,69.315162,1713.600303,427.591718
min,-80995.000000,0.000000,12346.000000,-168469.600000
25%,2.000000,1.250000,13953.000000,4.200000
50%,5.000000,1.950000,15152.000000,11.100000
75%,12.000000,3.750000,16791.000000,19.500000
max,80995.000000,38970.000000,18287.000000,168469.600000


Ainsi, grâce à la colonne CA, on remarque qu'il y a effectivement des clients disposant de valeurs aberrantes (ex - valeur min : -168469)
<p style="color:#CB4919;">Nous allons regrouper la consommation (CA) par client afin de faire ressortir les clients qui nous paraissent aberrants<p>

In [49]:
from pandas import unique
client_conso_total = df.groupby(by=['CustomerID']).sum()
client_negatif =  client_conso_total[client_conso_total['CA'] <= 0].index
print("Nombre de clients aberrants en % : ",round(len(client_negatif)/len(unique(df['CustomerID'])),4)*100,"%")

Nombre de clients aberrants en % :  1.17 %


Nous avons définit ici, que les clients dit aberrants sont ceux disposant d'une consommation totale <label style="color:#CB4919;" >nulle ou négative</label><p>
Au vu du nombre faible de clients aberrants, nous avons pris la décision de les <label style="color:#CB4919;" >supprimer</label>

In [50]:
df = df.set_index("CustomerID").drop(client_negatif).reset_index()
df.count()

CustomerID     406494
InvoiceNo      406494
StockCode      406494
Description    406494
Quantity       406494
InvoiceDate    406494
UnitPrice      406494
Country        406494
CA             406494
dtype: int64

## 2.3 - Suppression des factures annulables et aberrantes

### 2.3.1 - Suppression des factures avec un prix unitaire nul

In [51]:
df[['CA','UnitPrice']].describe()

,CA,UnitPrice
count,406494.000000,406494.000000
mean,20.451212,3.433818
std,391.925336,68.917552
min,-168469.600000,0.000000
25%,4.200000,1.250000
50%,11.250000,1.950000
75%,19.500000,3.750000
max,168469.600000,38970.000000


On peut remarquer qu'il y a des factures disposant d'un prix unitaire nul

In [52]:
print("Nombre de factures à prix unitaire nul en % : ",round(len(df[df['UnitPrice'] == 0])/len(df),4)*100,"%")

Nombre de factures à prix unitaire nul en % :  0.01 %


Au vu du nombre faible de factures aberrantes, nous avons pris la décision de les <label style="color:#CB4919;" >supprimer</label>

### 2.3.2 - Suppression des factures dites "annulables"

In [53]:
index = ["C581484","C556445","581483","556444"]
exemple = df.set_index("InvoiceNo").loc[index]
exemple[['Description','CustomerID','Quantity','UnitPrice','CA']]

,Description,CustomerID,Quantity,UnitPrice,CA
InvoiceNo,,,,,
C581484,"PAPER CRAFT , LITTLE BIRDIE",16446,-80995,2.08,-168469.6
C556445,Manual,15098,-1,38970.00,-38970.0
581483,"PAPER CRAFT , LITTLE BIRDIE",16446,80995,2.08,168469.6
556444,PICNIC BASKET WICKER 60 PIECES,15098,60,649.50,38970.0


In [54]:
print("Nombre de factures annulées : ",len([v for v in df['InvoiceNo'] if "C" in v]))

Nombre de factures annulées :  8695


Vu le nombre de factures annulées, cela peut entrainer des valeurs aberrantes pour les futurs traitement
<p>
    Il n'y a pas de règles générales qui permettrait d'annulées ces factures : 
    <p>
        Doublons, annulations qui correspond à plusieurs ou aucune facture, méthodes de calcules des montants changeantes etc 
<p>
<p>Cependant pour limiter leur impact, une règle correspondante à de nombreuses factures a été établi afin de les supprimer
<p style="color:#CB4919;">On declare une facture positive annulée si elle correspond à une facture négative par une égalité sur les colonnes suivantes : CustomerID, CA (en absolut), StockCode</p>

In [62]:
from pandas import concat, DataFrame
import warnings
warnings.filterwarnings("ignore")

df_index = df.reset_index().set_index('InvoiceNo')
liste_neg = list(set([inv for inv in df['InvoiceNo'] if "C" in inv]))
df_neg = df_index.loc[liste_neg,:].reset_index()
df_pos = df_index.drop(liste_neg).reset_index()
df_invoice_drop = DataFrame(columns=df_pos.columns)
av = 0
print("0%")
for inv,row in df_neg.iterrows() :   
    #temps correspond au nombre de factures positives égales à une facture négative
    temps = df_pos[(df_pos['CustomerID'] == row['CustomerID']) & (df_pos['CA'] == abs(row['CA'])) & (df_pos['StockCode'] == row['StockCode'])].values
    #s'il y a plusieurs factures positives correspondant, on supprime la première
    if len(temps) > 1:
        df_invoice_drop.loc[inv,:] = temps[0]
    elif temps == [] :
        df_invoice_drop.loc[inv,:] = None
    elif len(temps) == 1:
        df_invoice_drop.loc[inv,:] = temps
    #Etat d'avancement du traitement 
    t = av
    av = round((inv/len(df_neg))*100)+1
    if av%10 == 0 and t != av:
        print(av,"%")

liste_drop_pos = df_invoice_drop.dropna()
liste_drop_neg = df_neg.loc[liste_drop_pos.index,:]
liste_val_drop = concat([liste_drop_neg,liste_drop_pos]).set_index('index')
df_traitement = df.drop(liste_val_drop.index)

print("Pourcentage de factures annulées: ",round((len(liste_drop_neg)/len(df_neg)),4)*100,"%")

0%
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %
100 %
Pourcentage de factures annulées:  35.17 %


In [63]:
df_traitement.to_csv(r'C:\Users\arman\Desktop\data\Online_Retail_traitemant.csv',sep=";",index=False)